# TP3-Projeto

## Import do dataset

Imports iniciais

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from PIL import Image
import os
from tensorflow.keras.utils import to_categorical
import random
random.seed(0)

import collections
from sklearn import preprocessing
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import time
from sklearn import metrics
from IPython.display import Markdown, display
import random
import matplotlib.pyplot as plt

Import e tratamento do dataset

In [2]:
PATH = 'C:/Users/HyBr1d/Desktop/Faculdade/3º Ano/1º Semestre/IC/Trabalho/Meta3/notMNIST_small/notMNIST_small'

classes = os.listdir(PATH)
num_classes = len(classes)

print("There are {} classes: {}".format(num_classes, classes))

X = []
y = []

for directory in os.listdir(PATH):
    for image in os.listdir(PATH + '/' + directory):
        try:
            path = PATH + '/' + directory + '/' + image
            img = Image.open(path)
            img.load()
            img_X = np.asarray(img, dtype=np.int16)
            X.append(img_X)
            y.append(directory)
        except:
            None
            
X = np.asarray(X)
y = np.asarray(y)

num_images = len(X)
size = len(X[0])

X.shape, y.shape

y = list(map(lambda x: ord(x) - ord('A'), y))
y = np.asarray(y)

indices = np.arange(X.shape[0])
np.random.shuffle(indices)

X = X[indices]
y = y[indices]

num_train_img = np.int(0.7 * X.shape[0])
num_train_img

X_train = X[:num_train_img]
y_train = y[:num_train_img]

X_test = X[num_train_img:]
y_test = y[num_train_img:]

if y_train.ndim == 1: y_train = to_categorical(y_train, 10)
if y_test.ndim == 1: y_test = to_categorical(y_test, 10)

if np.max(X_train) == 255: X_train = X_train / 255 
if np.max(X_test) == 255: X_test = X_test / 255

if X_train.ndim == 3:
    num_pixels = size * size
    X_train_1d = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
    X_test_1d = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

if X_train.ndim == 3:
    X_train = X_train.reshape(-1, size, size, 1).astype('float32')
    X_test = X_test.reshape(-1, size, size, 1).astype('float32')

X_train.shape, X_test.shape

input_shape = X_train.shape[1:]
input_shape

X_train_1d.shape, X_test_1d.shape

input_shape_1d = X_train_1d.shape[1]
input_shape_1d

y_train.shape, y_test.shape

There are 10 classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']


((13106, 10), (5618, 10))

## Import do Cuckoo search algorithm

In [3]:
from niapy.algorithms.basic import CuckooSearch
algorithm = CuckooSearch()
algorithm.set_parameters(pa=0.5)

## Testes

### N_estimators (20,100,20), Max_depth(2,20,2) e pa(0.5)

In [4]:
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': range(20, 100, 20),
    'max_depth': range(2, 20, 2),
}

clf = RandomForestClassifier(random_state=42)

from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV

nia_search = NatureInspiredSearchCV(
    clf,
    param_grid,
    cv=2,
    verbose=3,
    algorithm=algorithm,
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,    
    runs=1
)

nia_search.fit(X_train_1d, y_train)

from sklearn.metrics import classification_report
clf = RandomForestClassifier(**nia_search.best_params_, random_state=42)


clf.fit(X_train_1d, y_train)

y_pred = clf.predict(X_test_1d)

print(classification_report(y_test, y_pred, digits=4))

print('Melhor Parametro:\n', nia_search.best_params_)
print(nia_search.best_score_)


Fitting 2 folds for some of the 36 candidates, which might total in 72 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END ...................max_depth=8, n_estimators=80; total time=   5.0s
[CV 2/2] END ...................max_depth=8, n_estimators=80; total time=   4.7s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END ..................max_depth=12, n_estimators=60; total time=   4.3s
[CV 2/2] END ..................max_depth=12, n_estimators=60; total time=   4.7s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END ..................max_depth=16, n_estimators=40; total time=   3.4s
[CV 2/2] END ..................max_depth=16, n_estimators=40; total time=   3.4s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END ...................max_depth=2, n_estimators=20; total time=   0.4s
[CV 2/2] END ...................max_depth=2, n_estimators=20; total time=   0.3s
Fitting 2 folds for each of 1 candidates

C:\Users\HyBr1d\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    clf,
    param_grid,
    cv=2,
    verbose=3,
)

grid_search.fit(X_train_1d, y_train)


from sklearn.metrics import classification_report
clf = RandomForestClassifier(**grid_search.best_params_, random_state=42)


clf.fit(X_train_1d, y_train)

y_pred = clf.predict(X_test_1d)

print(classification_report(y_test, y_pred, digits=4))

print('Melhor Parametro:\n', grid_search.best_params_)
print('Melhor accuracy:\n',grid_search.best_score_)

Fitting 2 folds for each of 36 candidates, totalling 72 fits
[CV 1/2] END ...................max_depth=2, n_estimators=20; total time=   0.4s
[CV 2/2] END ...................max_depth=2, n_estimators=20; total time=   0.4s
[CV 1/2] END ...................max_depth=2, n_estimators=40; total time=   0.8s
[CV 2/2] END ...................max_depth=2, n_estimators=40; total time=   0.8s
[CV 1/2] END ...................max_depth=2, n_estimators=60; total time=   1.2s
[CV 2/2] END ...................max_depth=2, n_estimators=60; total time=   1.2s
[CV 1/2] END ...................max_depth=2, n_estimators=80; total time=   1.8s
[CV 2/2] END ...................max_depth=2, n_estimators=80; total time=   1.7s
[CV 1/2] END ...................max_depth=4, n_estimators=20; total time=   0.6s
[CV 2/2] END ...................max_depth=4, n_estimators=20; total time=   0.6s
[CV 1/2] END ...................max_depth=4, n_estimators=40; total time=   1.3s
[CV 2/2] END ...................max_depth=4, n_e

C:\Users\HyBr1d\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### N_estimators (20,100,20), Max_depth(2,20,2) e pa(0.1)

In [6]:
algorithm = CuckooSearch()
algorithm.set_parameters(pa=0.1)
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': range(20, 100, 20),
    'max_depth': range(2, 20, 2),
}

clf = RandomForestClassifier(random_state=42)

from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV

nia_search = NatureInspiredSearchCV(
    clf,
    param_grid,
    cv=2,
    verbose=3,
    algorithm=algorithm,
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,    
    runs=1,
    n_jobs=-1
)

nia_search.fit(X_train_1d, y_train)

from sklearn.metrics import classification_report
clf = RandomForestClassifier(**nia_search.best_params_, random_state=42)


clf.fit(X_train_1d, y_train)

y_pred = clf.predict(X_test_1d)

print(classification_report(y_test, y_pred, digits=4))

print('Melhor Parametro:\n', nia_search.best_params_)
print(nia_search.best_score_)

Fitting 2 folds for some of the 36 candidates, which might total in 72 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidate

C:\Users\HyBr1d\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    clf,
    param_grid,
    cv=2,
    verbose=3,
    n_jobs=-1
)

grid_search.fit(X_train_1d, y_train)


from sklearn.metrics import classification_report
clf = RandomForestClassifier(**grid_search.best_params_, random_state=42)


clf.fit(X_train_1d, y_train)

y_pred = clf.predict(X_test_1d)

print(classification_report(y_test, y_pred, digits=4))

print('Melhor Parametro:\n', grid_search.best_params_)
print('Melhor accuracy:\n',grid_search.best_score_)

Fitting 2 folds for each of 36 candidates, totalling 72 fits
              precision    recall  f1-score   support

           0     0.9978    0.8053    0.8913       565
           1     0.9953    0.7326    0.8440       576
           2     0.9915    0.8345    0.9062       556
           3     0.9977    0.7856    0.8790       555
           4     0.9908    0.7688    0.8658       558
           5     0.9917    0.8856    0.9357       542
           6     0.9861    0.7933    0.8793       537
           7     0.9978    0.7986    0.8871       561
           8     0.9821    0.8003    0.8819       616
           9     0.9734    0.8623    0.9145       552

   micro avg     0.9902    0.8062    0.8887      5618
   macro avg     0.9904    0.8067    0.8885      5618
weighted avg     0.9904    0.8062    0.8881      5618
 samples avg     0.8062    0.8062    0.8062      5618

Melhor Parametro:
 {'max_depth': 12, 'n_estimators': 80}
Melhor accuracy:
 0.7920036624446818


C:\Users\HyBr1d\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### N_estimators (100,200,20), Max_depth(20,40,2) e pa(0.5)

In [8]:
algorithm = CuckooSearch()
algorithm.set_parameters(pa=0.5)
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': range(100, 200, 20),
    'max_depth': range(20, 40, 2),
}

clf = RandomForestClassifier(random_state=42)

from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV

nia_search = NatureInspiredSearchCV(
    clf,
    param_grid,
    cv=2,
    verbose=3,
    algorithm=algorithm,
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,    
    runs=1,
    n_jobs=-1
)

nia_search.fit(X_train_1d, y_train)

from sklearn.metrics import classification_report
clf = RandomForestClassifier(**nia_search.best_params_, random_state=42)


clf.fit(X_train_1d, y_train)

y_pred = clf.predict(X_test_1d)

print(classification_report(y_test, y_pred, digits=4))

print('Melhor Parametro:\n', nia_search.best_params_)
print(nia_search.best_score_)

Fitting 2 folds for some of the 50 candidates, which might total in 100 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidat

C:\Users\HyBr1d\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    clf,
    param_grid,
    cv=2,
    verbose=3
)

grid_search.fit(X_train_1d, y_train)


from sklearn.metrics import classification_report
clf = RandomForestClassifier(**grid_search.best_params_, random_state=42)


clf.fit(X_train_1d, y_train)

y_pred = clf.predict(X_test_1d)

print(classification_report(y_test, y_pred, digits=4))

print('Melhor Parametro:\n', grid_search.best_params_)
print('Melhor accuracy:\n',grid_search.best_score_)

Fitting 2 folds for each of 50 candidates, totalling 100 fits
[CV 1/2] END .................max_depth=20, n_estimators=100; total time=   6.2s
[CV 2/2] END .................max_depth=20, n_estimators=100; total time=   6.5s
[CV 1/2] END .................max_depth=20, n_estimators=120; total time=   7.8s
[CV 2/2] END .................max_depth=20, n_estimators=120; total time=   7.9s
[CV 1/2] END .................max_depth=20, n_estimators=140; total time=  10.0s
[CV 2/2] END .................max_depth=20, n_estimators=140; total time=  10.5s
[CV 1/2] END .................max_depth=20, n_estimators=160; total time=  11.0s
[CV 2/2] END .................max_depth=20, n_estimators=160; total time=  10.3s
[CV 1/2] END .................max_depth=20, n_estimators=180; total time=  11.9s
[CV 2/2] END .................max_depth=20, n_estimators=180; total time=  11.7s
[CV 1/2] END .................max_depth=22, n_estimators=100; total time=   6.8s
[CV 2/2] END .................max_depth=22, n_e

C:\Users\HyBr1d\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### N_estimators (100,200,20), Max_depth(2,20,2) e pa(0.5)

In [32]:
algorithm = CuckooSearch()
algorithm.set_parameters(pa=0.5)
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': range(100, 200, 20),
    'max_depth': range(2, 20, 2),
}

clf = RandomForestClassifier(random_state=42)

from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV

nia_search = NatureInspiredSearchCV(
    clf,
    param_grid,
    cv=2,
    verbose=3,
    algorithm=algorithm,
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,    
    runs=1,
    n_jobs=-1
)

nia_search.fit(X_train_1d, y_train)

from sklearn.metrics import classification_report
clf = RandomForestClassifier(**nia_search.best_params_, random_state=42)


clf.fit(X_train_1d, y_train)

y_pred = clf.predict(X_test_1d)

print(classification_report(y_test, y_pred, digits=4))

print('Melhor Parametro:\n', nia_search.best_params_)
print(nia_search.best_score_)

Fitting 2 folds for some of the 45 candidates, which might total in 90 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidate

C:\Users\HyBr1d\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    clf,
    param_grid,
    cv=2,
    verbose=3
)

grid_search.fit(X_train_1d, y_train)


from sklearn.metrics import classification_report
clf = RandomForestClassifier(**grid_search.best_params_, random_state=42)


clf.fit(X_train_1d, y_train)

y_pred = clf.predict(X_test_1d)

print(classification_report(y_test, y_pred, digits=4))

print('Melhor Parametro:\n', grid_search.best_params_)
print('Melhor accuracy:\n',grid_search.best_score_)

Fitting 2 folds for each of 45 candidates, totalling 90 fits
[CV 1/2] END ..................max_depth=2, n_estimators=100; total time=   1.8s
[CV 2/2] END ..................max_depth=2, n_estimators=100; total time=   1.8s
[CV 1/2] END ..................max_depth=2, n_estimators=120; total time=   2.2s
[CV 2/2] END ..................max_depth=2, n_estimators=120; total time=   2.1s
[CV 1/2] END ..................max_depth=2, n_estimators=140; total time=   2.6s
[CV 2/2] END ..................max_depth=2, n_estimators=140; total time=   2.5s
[CV 1/2] END ..................max_depth=2, n_estimators=160; total time=   2.9s
[CV 2/2] END ..................max_depth=2, n_estimators=160; total time=   2.9s
[CV 1/2] END ..................max_depth=2, n_estimators=180; total time=   3.2s
[CV 2/2] END ..................max_depth=2, n_estimators=180; total time=   3.5s
[CV 1/2] END ..................max_depth=4, n_estimators=100; total time=   2.9s
[CV 2/2] END ..................max_depth=4, n_es

C:\Users\HyBr1d\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### N_estimators (150,190,5), Max_depth(10,14,1) e pa(0.5)

In [22]:
algorithm = CuckooSearch()
algorithm.set_parameters(pa=0.5)
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': range(150,190,5),
    'max_depth': range(10,14,1),
}

clf = RandomForestClassifier(random_state=42)

from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV

nia_search = NatureInspiredSearchCV(
    clf,
    param_grid,
    cv=2,
    verbose=3,
    algorithm=algorithm,
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,    
    runs=1,
    n_jobs=-1
)

nia_search.fit(X_train_1d, y_train)

from sklearn.metrics import classification_report
clf = RandomForestClassifier(**nia_search.best_params_, random_state=42)


clf.fit(X_train_1d, y_train)

y_pred = clf.predict(X_test_1d)

print(classification_report(y_test, y_pred, digits=4))

print('Melhor Parametro:\n', nia_search.best_params_)
print(nia_search.best_score_)

Fitting 2 folds for some of the 32 candidates, which might total in 64 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidate

C:\Users\HyBr1d\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### N_estimators (150,190,5), Max_depth(10,14,1) e pa(0.1)

In [23]:
algorithm = CuckooSearch()
algorithm.set_parameters(pa=0.1)
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': range(150,190,5),
    'max_depth': range(10,14,1),
}

clf = RandomForestClassifier(random_state=42)

from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV

nia_search = NatureInspiredSearchCV(
    clf,
    param_grid,
    cv=2,
    verbose=3,
    algorithm=algorithm,
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,    
    runs=1,
    n_jobs=-1
)

nia_search.fit(X_train_1d, y_train)

from sklearn.metrics import classification_report
clf = RandomForestClassifier(**nia_search.best_params_, random_state=42)


clf.fit(X_train_1d, y_train)

y_pred = clf.predict(X_test_1d)

print(classification_report(y_test, y_pred, digits=4))

print('Melhor Parametro:\n', nia_search.best_params_)
print(nia_search.best_score_)

Fitting 2 folds for some of the 32 candidates, which might total in 64 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidate

C:\Users\HyBr1d\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### N_estimators (150,190,5), Max_depth(10,14,1) e pa(0.2)

In [24]:
algorithm = CuckooSearch()
algorithm.set_parameters(pa=0.2)
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': range(150,190,5),
    'max_depth': range(10,14,1),
}

clf = RandomForestClassifier(random_state=42)

from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV

nia_search = NatureInspiredSearchCV(
    clf,
    param_grid,
    cv=2,
    verbose=3,
    algorithm=algorithm,
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,    
    runs=1,
    n_jobs=-1
)

nia_search.fit(X_train_1d, y_train)

from sklearn.metrics import classification_report
clf = RandomForestClassifier(**nia_search.best_params_, random_state=42)


clf.fit(X_train_1d, y_train)

y_pred = clf.predict(X_test_1d)

print(classification_report(y_test, y_pred, digits=4))

print('Melhor Parametro:\n', nia_search.best_params_)
print(nia_search.best_score_)

Fitting 2 folds for some of the 32 candidates, which might total in 64 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidate

C:\Users\HyBr1d\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### N_estimators (150,190,5), Max_depth(10,14,1) e pa(0.75)

In [25]:
algorithm = CuckooSearch()
algorithm.set_parameters(pa=0.75)
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': range(150,190,5),
    'max_depth': range(10,14,1),
}

clf = RandomForestClassifier(random_state=42)

from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV

nia_search = NatureInspiredSearchCV(
    clf,
    param_grid,
    cv=2,
    verbose=3,
    algorithm=algorithm,
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,    
    runs=1,
    n_jobs=-1
)

nia_search.fit(X_train_1d, y_train)

from sklearn.metrics import classification_report
clf = RandomForestClassifier(**nia_search.best_params_, random_state=42)


clf.fit(X_train_1d, y_train)

y_pred = clf.predict(X_test_1d)

print(classification_report(y_test, y_pred, digits=4))

print('Melhor Parametro:\n', nia_search.best_params_)
print(nia_search.best_score_)

Fitting 2 folds for some of the 32 candidates, which might total in 64 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidate

C:\Users\HyBr1d\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### N_estimators (150,190,5), Max_depth(10,14,1) e pa(0.5) e population_size(100)

In [26]:
algorithm = CuckooSearch()
algorithm.set_parameters(pa=0.5)
algorithm.set_parameters(population_size=100)
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': range(150,190,5),
    'max_depth': range(10,14,1),
}

clf = RandomForestClassifier(random_state=42)

from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV

nia_search = NatureInspiredSearchCV(
    clf,
    param_grid,
    cv=2,
    verbose=3,
    algorithm=algorithm,
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,    
    runs=1,
    n_jobs=-1
)

nia_search.fit(X_train_1d, y_train)

from sklearn.metrics import classification_report
clf = RandomForestClassifier(**nia_search.best_params_, random_state=42)


clf.fit(X_train_1d, y_train)

y_pred = clf.predict(X_test_1d)

print(classification_report(y_test, y_pred, digits=4))

print('Melhor Parametro:\n', nia_search.best_params_)
print(nia_search.best_score_)

Fitting 2 folds for some of the 32 candidates, which might total in 64 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidate

C:\Users\HyBr1d\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    clf,
    param_grid,
    cv=2,
    verbose=3
)

grid_search.fit(X_train_1d, y_train)


from sklearn.metrics import classification_report
clf = RandomForestClassifier(**grid_search.best_params_, random_state=42)


clf.fit(X_train_1d, y_train)

y_pred = clf.predict(X_test_1d)

print(classification_report(y_test, y_pred, digits=4))

print('Melhor Parametro:\n', grid_search.best_params_)
print('Melhor accuracy:\n',grid_search.best_score_)

Fitting 2 folds for each of 32 candidates, totalling 64 fits
[CV 1/2] END .................max_depth=10, n_estimators=150; total time=   9.2s
[CV 2/2] END .................max_depth=10, n_estimators=150; total time=   8.7s
[CV 1/2] END .................max_depth=10, n_estimators=155; total time=   9.5s
[CV 2/2] END .................max_depth=10, n_estimators=155; total time=   8.8s
[CV 1/2] END .................max_depth=10, n_estimators=160; total time=   9.6s
[CV 2/2] END .................max_depth=10, n_estimators=160; total time=   9.6s
[CV 1/2] END .................max_depth=10, n_estimators=165; total time=  10.1s
[CV 2/2] END .................max_depth=10, n_estimators=165; total time=   9.4s
[CV 1/2] END .................max_depth=10, n_estimators=170; total time=  10.1s
[CV 2/2] END .................max_depth=10, n_estimators=170; total time=   9.9s
[CV 1/2] END .................max_depth=10, n_estimators=175; total time=  10.3s
[CV 2/2] END .................max_depth=10, n_es

C:\Users\HyBr1d\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### N_estimators (170,180,1), Max_depth(10,14,1) e pa(0.5)

In [34]:
algorithm = CuckooSearch()
algorithm.set_parameters(pa=0.5)
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': range(170,180,1),
    'max_depth': range(12,14,1),
}

clf = RandomForestClassifier(random_state=42)

from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV

nia_search = NatureInspiredSearchCV(
    clf,
    param_grid,
    cv=2,
    verbose=3,
    algorithm=algorithm,
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=20,    
    runs=1,
    n_jobs=-1
)

nia_search.fit(X_train_1d, y_train)

from sklearn.metrics import classification_report
clf = RandomForestClassifier(**nia_search.best_params_, random_state=42)


clf.fit(X_train_1d, y_train)

y_pred = clf.predict(X_test_1d)

print(classification_report(y_test, y_pred, digits=4))

print('Melhor Parametro:\n', nia_search.best_params_)
print(nia_search.best_score_)

Fitting 2 folds for some of the 20 candidates, which might total in 40 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidate

C:\Users\HyBr1d\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    clf,
    param_grid,
    cv=2,
    verbose=3
)

grid_search.fit(X_train_1d, y_train)


from sklearn.metrics import classification_report
clf = RandomForestClassifier(**grid_search.best_params_, random_state=42)


clf.fit(X_train_1d, y_train)

y_pred = clf.predict(X_test_1d)

print(classification_report(y_test, y_pred, digits=4))

print('Melhor Parametro:\n', grid_search.best_params_)
print('Melhor accuracy:\n',grid_search.best_score_)

Fitting 2 folds for each of 20 candidates, totalling 40 fits
[CV 1/2] END .................max_depth=12, n_estimators=170; total time=  10.9s
[CV 2/2] END .................max_depth=12, n_estimators=170; total time=  10.5s
[CV 1/2] END .................max_depth=12, n_estimators=171; total time=  11.2s
[CV 2/2] END .................max_depth=12, n_estimators=171; total time=  11.2s
[CV 1/2] END .................max_depth=12, n_estimators=172; total time=  11.1s
[CV 2/2] END .................max_depth=12, n_estimators=172; total time=  10.8s
[CV 1/2] END .................max_depth=12, n_estimators=173; total time=  10.8s
[CV 2/2] END .................max_depth=12, n_estimators=173; total time=  11.0s
[CV 1/2] END .................max_depth=12, n_estimators=174; total time=  10.8s
[CV 2/2] END .................max_depth=12, n_estimators=174; total time=  11.4s
[CV 1/2] END .................max_depth=12, n_estimators=175; total time=  10.9s
[CV 2/2] END .................max_depth=12, n_es

C:\Users\HyBr1d\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
